## **install pyspark on google colab with TPU**

In [0]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u191-b12-0ubuntu0.18.04.1).
openjdk-8-jre-headless set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
    100% |████████████████████████████████| 213.4MB 51kB/s 
    100% |████████████████████████████████| 204kB 30.4MB/s 
  Running setup.py bdist_wheel for pyspark ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark


I plan to build recommendation system on google gcloud. I want to make sure all codes are good before gcloud. Thus, I set up spark in google colab with TPU.

The data and codes are originally from: https://github.com/GoogleCloudPlatform/spark-recommendation-engine

**Part 1**: **grid search for best parameters in recommender model**

In [0]:
rdd = sc.parallelize(["Hello Spark"])
counts = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
print(counts)

[('Hello', 1), ('Spark', 1)]


In [0]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import Bucketizer, MinMaxScaler, VectorAssembler
from pyspark.sql.types import DoubleType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml import Transformer
from pyspark.sql import Window

In [0]:
from google.colab import files
files.upload()

Saving ratings.csv to ratings.csv


{'ratings.csv': b'10,1,1\n18,1,2\n13,1,1\n7,2,2\n4,2,2\n13,2,3\n19,2,2\n12,2,1\n11,2,1\n1,2,2\n20,2,2\n2,2,4\n3,2,1\n0,3,4\n4,3,5\n8,3,4\n7,3,4\n10,3,5\n16,3,5\n21,3,5\n23,3,5\n21,4,2\n4,4,1\n7,4,2\n23,4,1\n24,4,2\n6,4,1\n3,4,1\n13,4,1\n2,4,1\n5,4,3\n18,4,4\n15,4,4\n22,4,3\n1,4,4\n17,4,3\n20,4,1\n19,4,3\n9,4,1\n14,4,1\n12,4,3\n11,4,4\n16,5,1\n10,5,1\n23,5,2\n21,5,2\n4,5,1\n11,5,3\n6,5,1\n12,5,4\n15,5,1\n13,5,4\n17,5,4\n5,5,3\n2,5,1\n3,5,1\n9,5,4\n1,5,3\n20,5,3\n19,5,3\n14,5,1\n24,5,2\n18,5,2\n0,6,5\n4,6,5\n8,6,5\n7,6,5\n10,6,5\n16,6,5\n21,6,4\n23,6,5\n16,7,1\n19,7,2\n14,7,1\n1,7,1\n22,7,3\n17,7,2\n12,7,1\n9,7,3\n6,7,3\n24,7,4\n5,7,4\n15,7,3\n13,7,4\n3,7,2\n2,7,1\n20,7,4\n11,7,4\n18,7,3\n21,8,1\n16,8,1\n7,8,2\n10,8,2\n8,8,1\n23,8,1\n22,8,2\n6,8,2\n18,8,4\n13,8,4\n12,8,3\n5,8,2\n1,8,3\n15,8,1\n3,8,2\n17,8,4\n2,8,4\n14,8,3\n20,8,3\n24,8,1\n19,8,2\n11,8,2\n9,8,1\n10,9,1\n7,9,1\n21,9,2\n4,9,2\n8,9,2\n16,9,1\n23,9,1\n15,9,2\n21,10,2\n4,10,1\n12,10,3\n18,10,1\n21,11,1\n23,11,1\n7,11,1\n16,11,

In [0]:
df_accommodations = spark.read.csv('accommodations.csv')


In [0]:
dfRates = spark.read.csv('ratings.csv')

In [0]:
dfRates.show()

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
| 10|  1|  1|
| 18|  1|  2|
| 13|  1|  1|
|  7|  2|  2|
|  4|  2|  2|
| 13|  2|  3|
| 19|  2|  2|
| 12|  2|  1|
| 11|  2|  1|
|  1|  2|  2|
| 20|  2|  2|
|  2|  2|  4|
|  3|  2|  1|
|  0|  3|  4|
|  4|  3|  5|
|  8|  3|  4|
|  7|  3|  4|
| 10|  3|  5|
| 16|  3|  5|
| 21|  3|  5|
+---+---+---+
only showing top 20 rows



In [0]:
dfRates = spark.read.csv('ratings.csv',header=True, inferSchema=True).cache()

In [0]:
dfRates.show(5)

+---+---+---+
| 10| 11| 12|
+---+---+---+
| 18|  1|  2|
| 13|  1|  1|
|  7|  2|  2|
|  4|  2|  2|
| 13|  2|  3|
+---+---+---+
only showing top 5 rows



In [0]:
#df = df.withColumnRenamed("colName", "newColName")
#       .withColumnRenamed("colName2", "newColName2")


dfRates = dfRates.withColumnRenamed("10", "userId")\
.withColumnRenamed("11", "accoId")\
.withColumnRenamed("12", "rating")

In [0]:
dfRates.show(5)


+------+------+------+
|userId|accoId|rating|
+------+------+------+
|    18|     1|     2|
|    13|     1|     1|
|     7|     2|     2|
|     4|     2|     2|
|    13|     2|     3|
+------+------+------+
only showing top 5 rows



In [0]:
def howFarAreWe(model, against, sizeAgainst):
  # Ignore the rating column
  againstNoRatings = against.map(lambda x: (int(x[0]), int(x[1])) )

  # Keep the rating to compare against
  againstWiRatings = against.map(lambda x: ((int(x[0]),int(x[1])), int(x[2])) )

  # Make a prediction and map it for later comparison
  # The map has to be ((user,product), rating) not ((product,user), rating)
  predictions = model.predictAll(againstNoRatings).map(lambda p: ( (p[0],p[1]), p[2]) )

  # Returns the pairs (prediction, rating)
  predictionsAndRatings = predictions.join(againstWiRatings).values()

  # Returns the variance
  return sqrt(predictionsAndRatings.map(lambda s: (s[0] - s[1]) ** 2).reduce(add) / float(sizeAgainst))

In [0]:
rddUserRatings = dfRates.filter(dfRates.userId == 0).rdd
print(rddUserRatings.count())

9


In [0]:
rddRates = dfRates.rdd
rddTraining, rddValidating, rddTesting = rddRates.randomSplit([6,2,2])

In [0]:
rddTraining.union(rddUserRatings)
nbValidating = rddValidating.count()
nbTesting    = rddTesting.count()

In [0]:
print("Training: %d, validation: %d, test: %d" % (rddTraining.count(), nbValidating, rddTesting.count()))

Training: 720, validation: 228, test: 237


In [0]:
ranks  = [5,10,15,20]
reguls = [0.1, 1,10]
iters  = [5,10,20]

finalModel = None
finalRank  = 0
finalRegul = float(0)
finalIter  = -1
finalDist   = float(100)


In [0]:
from __future__ import print_function

import sys
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [0]:
for cRank, cRegul, cIter in itertools.product(ranks, reguls, iters):

  model = ALS.train(rddTraining, cRank, cIter, float(cRegul))
  dist = howFarAreWe(model, rddValidating, nbValidating)
  if dist < finalDist:
    print("Best so far:%f" % dist)
    finalModel = model
    finalRank  = cRank
    finalRegul = cRegul
    finalIter  = cIter
    finalDist  = dist

print("Rank %i" % finalRank)
print("Regul %f" % finalRegul)
print("Iter %i" % finalIter)
print("Dist %f" % finalDist)



Best so far:1.306835
Best so far:1.267861
Best so far:1.232597
Rank 10
Regul 0.100000
Iter 5
Dist 1.232597
